# Postgres Examples

In [ ]:
# Auth
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
postgres_db = {'drivername': 'postgresql',
               'username': 'postgres',
               'password': 'XM8(eTha++ma<(@C',
               'host': '34.71.18.220',
               'port': 5432,
               'database': 'deeptendies_sandbox'}
pgURL = URL(**postgres_db)
engine = create_engine(pgURL)

import pandas as pd
schema = 'finnhub_api_ingestion_stock_candles'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


```python
# show all schemas within a db
query = f"""
select schema_name
from information_schema.schemata;
"""
pd.read_sql(query, con=engine)

# select a specific table
table_name = 'finnhub_api_ingestion_earning_calendar_earning_calendar'
pd.read_sql(f"SELECT * FROM \"{schema}\".\"{table_name}\"",
                    con=engine)

# Show all tables within a schema
query = f"""
SELECT * FROM information_schema.tables 
WHERE table_schema = '{schema}' 
"""
pd.read_sql(query, con=engine)[['table_catalog', 'table_schema', 'table_name']].to_dict()

# select a specific table
table_name = 'finnhub_api_ingestion_earning_calendar_earning_calendar'
pd.read_sql(f"SELECT * FROM \"{schema}\".\"{table_name}\"",
                    con=engine)

# transformation
from datetime import date
today = date.today().strftime("%Y-%m-%d")
df = pd.read_sql(f"SELECT * FROM \"{schema}\".\"{table_name}\"",
                    con=engine)
df = df[df.date > today].sort_values(['date'],ascending = True).head(50).sort_values(['epsEstimate'],ascending = False)
topic_name = 'finnhub_api_ingestion_earning_calendar_earning_calendar' + "_sorted"

# to sql
df.to_sql(name=topic_name,
            con=engine,
            schema=schema,
            if_exists='append',
            index=False,
            method='multi')                    
```

# BigQuery Examples
https://colab.research.google.com/notebooks/bigquery.ipynb

```python
# Auth
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

project_id = 'deeptendies-platform'
from google.cloud import bigquery
client = bigquery.Client(project=project_id)
```

read / write

```python
import pandas as pd

# Write to BQ
pd.io.gbq.to_gbq(
    dataframe = df,
    destination_table = "deeptendies_dwh.finnhub_api_ingestion_earning_calendar_earning_calendar",
    project_id=project_id,
    if_exists = "replace"
)

# Read from BQ
df = pd.io.gbq.read_gbq("""
    SELECT * FROM `deeptendies-platform.deeptendies_dwh.finnhub_api_ingestion_earning_calendar_earning_calendar`
    LIMIT 1000
    """,
    project_id=project_id,
    dialect='standard')

df.head()
```

# Spark

In [ ]:
!pip install pyspark

In [ ]:
ticker ="TQQQ"

query = f"""
select *
from \"finnhub_api_ingestion_stock_candles\".\"{ticker}\";
"""
df = pd.read_sql(query, con=engine)

c: close, h: high, l: low, o: open, s: status, t: timestamp, v: volume

In [4]:
from pyspark.sql import SparkSession
import pandas as pd

def load_csv(TICKER: str):
    df = pd.read_csv("data/"+TICKER+".csv")
    spark = create_spark_session()
    sparkDF = spark.createDataFrame(df)
    sparkDF.printSchema()
    sparkDF.show(3)

def create_spark_session():
    spark = SparkSession.builder.master("local[1]").appName("ENSF612Project").getOrCreate()
    return spark

if __name__ == "__main__":
    df = load_csv("GOOGL")

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: long (nullable = true)



Py4JJavaError: An error occurred while calling o91.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (10.0.0.18 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.

hl_movement: high low move(difference between the highest and lowest point);
daily_movement: close value- open value;
hlm_divv:hl_movement/volume
daily_divv: daily_movement/volume


In [ ]:
import datetime
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType, StringType

print(df.columns)

@udf("string")
def get_date_udf(a):
    val = str(datetime.datetime.fromtimestamp(a).strftime('%Y-%m-%d %H:%M:%S'))
    return val

@udf("double")
def hl_movement(h,l):
    return h - l

@udf("double")
def daily_movement(o,c):
    return o - c

@udf("double")
def hlm_divv(m,v):
    return m /v

@udf("double")
def dailym_divv(m,v):
    return m /v

display(sparkDF.select("*", get_date_udf("t").alias("date")))

Index(['c', 'h', 'l', 'o', 's', 't', 'v'], dtype='object')


DataFrame[c: double, h: double, l: double, o: double, s: string, t: bigint, v: bigint, date: string]

# Feature engineering

In [ ]:
sdf = sparkDF.select("*",
                     get_date_udf("t").alias("date"),
                     hl_movement("h", "l").alias("hlmov"),
                     daily_movement("o","c").alias("daymov"),
                     ).sort("date")
sdf = sdf.select("*", hlm_divv("hlmov","c").alias("hlmovdivv"),
                 dailym_divv("daymov","c").alias("dmovdivv")
                 )
sdf.show()

+------+--------+--------+------+---+----------+--------+-------------------+------------------+--------------------+--------------------+--------------------+
|     c|       h|       l|     o|  s|         t|       v|               date|             hlmov|              daymov|           hlmovdivv|            dmovdivv|
+------+--------+--------+------+---+----------+--------+-------------------+------------------+--------------------+--------------------+--------------------+
| 71.08|   79.19|   70.74| 77.84| ok|1604880000|82932764|2020-11-09 00:00:00| 8.450000000000003|   6.760000000000005|  0.1188801350590884| 0.09510410804727075|
|67.245|  69.625|  65.175| 68.57| ok|1604966400|84333318|2020-11-10 00:00:00| 4.450000000000003|  1.3249999999999886| 0.06617592386051012| 0.01970406721689328|
|71.765|  72.095|   68.74|69.035| ok|1605052800|63415370|2020-11-11 00:00:00| 3.355000000000004|  -2.730000000000004| 0.04674980840242463|-0.03804082770152587|
|70.715|   73.46|   69.86|72.425| ok|160

In [ ]:
# converting back to pandas
df = sdf.toPandas().dropna()

In [ ]:
df[['dc', 'dh', 'dl','do', 'dv']]=df[[ 'c','h','l','o','v']].diff()

In [ ]:
df['date'] = pd.to_datetime(df["date"]).dt.date
df = df [df['date'] < datetime.date.today()]
df

,c,h,l,o,s,t,v,date,hlmov,daymov,hlmovdivv,dmovdivv,dc,dh,dl,do,dv
0,71.080,79.1900,70.740,77.840,ok,1604880000,82932764,2020-11-09,8.4500,6.760,0.118880,0.095104,NaN,NaN,NaN,NaN,NaN
1,67.245,69.6250,65.175,68.570,ok,1604966400,84333318,2020-11-10,4.4500,1.325,0.066176,0.019704,-3.835,-9.5650,-5.565,-9.270,1400554.0
2,71.765,72.0950,68.740,69.035,ok,1605052800,63415370,2020-11-11,3.3550,-2.730,0.046750,-0.038041,4.520,2.4700,3.565,0.465,-20917948.0
3,70.715,73.4600,69.860,72.425,ok,1605139200,71928550,2020-11-12,3.6000,1.710,0.050909,0.024182,-1.050,1.3650,1.120,3.390,8513180.0
4,72.570,73.0400,70.330,71.890,ok,1605225600,52448626,2020-11-13,2.7100,-0.680,0.037343,-0.009370,1.855,-0.4200,0.470,-0.535,-19479924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,158.780,159.3700,155.960,156.025,ok,1635811200,30042809,2021-11-02,3.4100,-2.755,0.021476,-0.017351,2.090,2.4400,2.780,0.225,-2468558.0
248,163.820,164.3200,158.070,159.340,ok,1635897600,36302613,2021-11-03,6.2500,-4.480,0.038152,-0.027347,5.040,4.9500,2.110,3.315,6259804.0
249,169.930,171.3477,164.730,165.430,ok,1635984000,40219862,2021-11-04,6.6177,-4.500,0.038944,-0.026481,6.110,7.0277,6.660,6.090,3917249.0
250,170.560,173.6600,168.730,172.460,ok,1636070400,42626357,2021-11-05,4.9300,1.900,0.028905,0.011140,0.630,2.3123,4.000,7.030,2406495.0


In [ ]:
!pip install autokeras

In [ ]:
import pandas as pd
import tensorflow as tf
import autokeras as ak

In [ ]:
df.columns

Index(['c', 'h', 'l', 'o', 's', 't', 'v', 'date', 'hlmov', 'daymov',
       'hlmovdivv', 'dmovdivv', 'dc', 'dh', 'dl', 'do', 'dv'],
      dtype='object')

In [ ]:
# We're using deep learning algorithm, LSTM, to compare with the algorithms from finRL library 
# Using autokeras timeseries forecaster
# https://autokeras.com/tutorial/timeseries_forecaster/
dataset = df.dropna()
dataset = dataset.replace(",", ".", regex=True)

val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

x_cols =     ['v', 'hlmov', 'daymov',
       'dc', 
    #    'dh',
        'dl', 'do', 'dv']

ycols = ['dh']

data_x = data_train[
    x_cols
].astype("float64")

data_x_val = validation_data[
    x_cols
].astype("float64")

# Data with train data and the unseen data from subsequent time steps.
data_x_test = dataset[
    x_cols
].astype("float64")

data_y = data_train[ycols].astype("float64")

data_y_val = validation_data[ycols].astype("float64")

print(data_x.shape)  # (6549, 12)
print(data_y.shape)  # (6549,)

(175, 7)
(175, 1)


In [ ]:
predict_from = 1
predict_until = 10
lookback = 10
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    # max_trials=1,
    objective="val_loss",
)
# Train the TimeSeriesForecaster with train data
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
    # epochs=10,
)

Trial 15 Complete [00h 07m 00s]
val_loss: 7.765697002410889

Best val_loss So Far: 6.567526340484619
Total elapsed time: 00h 20m 39s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


In [ ]:
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))
print("next 10 forecast dh:")
print(predictions)

8/8 [==============================] - 0s 13ms/step
(10, 1)
3/3 [==============================] - 2s 9ms/step - loss: 6.5675 - mean_squared_error: 6.5675
[6.567526340484619, 6.567526340484619]
next 10 forecast dh:
[[ 0.8707507 ]
 [ 1.0773495 ]
 [ 0.43577257]
 [ 1.01952   ]
 [ 1.2325165 ]
 [ 1.2308419 ]
 [ 0.4431724 ]
 [-0.44880378]
 [ 0.5153476 ]
 [-0.4437851 ]]


# dl

In [ ]:
# Using autokeras timeseries forecaster
# https://autokeras.com/tutorial/timeseries_forecaster/
dataset = df.dropna()
dataset = dataset.replace(",", ".", regex=True)

val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

x_cols =     ['v', 'hlmov', 'daymov',
       'dc', 
       'dh',
        # 'dl',
         'do', 'dv']

ycols = ['dl']

data_x = data_train[
    x_cols
].astype("float64")

data_x_val = validation_data[
    x_cols
].astype("float64")

# Data with train data and the unseen data from subsequent time steps.
data_x_test = dataset[
    x_cols
].astype("float64")

data_y = data_train[ycols].astype("float64")

data_y_val = validation_data[ycols].astype("float64")

print(data_x.shape)  # (6549, 12)
print(data_y.shape)  # (6549,)

(175, 7)
(175, 1)


In [ ]:
predict_from = 1
predict_until = 10
lookback = 10
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    # max_trials=1,
    objective="val_loss",
)
# Train the TimeSeriesForecaster with train data
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
    # epochs=10,
)

INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


In [ ]:
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))
print("next 10 forecast dl:")
print(predictions)

8/8 [==============================] - 0s 9ms/step
(10, 1)
3/3 [==============================] - 2s 9ms/step - loss: 11.1515 - mean_squared_error: 11.1515
[11.151470184326172, 11.151470184326172]
next 10 forecast dl:
[[ 0.8707507 ]
 [ 1.0773495 ]
 [ 0.43577257]
 [ 1.01952   ]
 [ 1.2325165 ]
 [ 1.2308419 ]
 [ 0.4431724 ]
 [-0.44880378]
 [ 0.5153476 ]
 [-0.4437851 ]]
